# Russian Synodal Bible (1885) and Primary Text Text Processing

## Import

### Libraries

In [ ]:
import os
import json
import time
import xml.etree.ElementTree as ET
import codecs
import pandas as pd 
import natasha
import re

### Definitions

In [ ]:
texts = './texts/fiction/'
bibleXML = './texts/bible/synodal.xml'
libCols = ['author','date','title','text']
tokenOHCO = ['title','part_num','para_num', 'sent_num', 'token_num']
bibleOHCO = ['book','chap_num','verse_num']

## Primary Texts

### Library

In [ ]:
libDf = pd.DataFrame(columns = libCols)
for t in os.listdir(texts): 
    if t[-4:] == '.txt': 
        #print(t)
        info = re.match(r'(\w+)-(\d{4})-(.+).txt', t)
        with codecs.open(texts+t, 'r', encoding='windows-1251') as f: 
            textytext = f.read()
        libDf = libDf.append({
            'author': info.group(1),
            'date': int(info.group(2)), 
            'title': info.group(3), 
            'text': textytext
        }, ignore_index=True)
        
libDf = libDf.sort_values(libCols[1:3]).set_index(libCols[2])
tokenDf = libDf[[libCols[3]]]
libDf = libDf.drop(columns=[libCols[3]])
libDf

### Tokens

In [ ]:
tokenDf = tokenDf.text.str.split('\n\n\n', expand=True).stack().to_frame()
tokenDf.index.names = tokenOHCO[:2]
tokenDf = tokenDf.rename(columns={0:'parts'})
tokenDf = tokenDf.parts.str.split('\n\s*\n', expand=True).stack().to_frame()
tokenDf.index.names = tokenOHCO[:3]
tokenDf = tokenDf.rename(columns={0:'paras'})
tokenDf = tokenDf.paras.str.split(r'!|\?|\.|;', expand=True).stack().to_frame()
tokenDf.index.names = tokenOHCO[:4]
tokenDf = tokenDf.rename(columns={0:'sents'})
sentDf = tokenDf
tokenDf = tokenDf.sents.str.split(r'\n|\s+', expand=True).stack().to_frame()
tokenDf.index.names = tokenOHCO
tokenDf = tokenDf.rename(columns={0:'token'})
tokenDf = tokenDf.token.apply(lambda x: x.lower().strip(r"\W\|\]\}\[\{\.\'\"\?;:,<>/1234567890")).to_frame()
tokenDf = tokenDf[tokenDf['token'].str.contains(r'\w+')]
tokenDf

In [ ]:
sentDf = sentDf[sentDf['sents'].str.contains(r'\w+')].sents.apply(lambda x: x.lower().strip(r"\W\n\|\]\}\[\{\.\'\"\?;:,<>/1234567890")).to_frame()
sentDf[sentDf['sents'].str.contains('r\w')]

## Bible

### XML

In [ ]:
xmltree = ET.parse(bibleXML)
xmlroot = xmltree.getroot()

### Create Bible Dictionary

In [ ]:
bibleDicts = []
for book in xmlroot: 
    name = book.attrib.get("n")
    for chapter in book: 
        chap_num = chapter.attrib.get("n")
        for verse in chapter: 
            verse_num = verse.attrib.get("n")
            verseDict = {'book': name, 
                         'chap_num': chap_num, 
                         'verse_num': verse_num,
                         'text': verse.text}
            bibleDicts.append(verseDict)

### Bible

In [ ]:
bibleDf = pd.DataFrame.from_dict(bibleDicts).set_index(bibleOHCO)
bibleDf